In [8]:
# import the required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel


In [16]:
# import the data
path = 'C:/Users/20202201/Documents/Uni/Master/Year2/Semester 1/Natural Language Processing/NLP/Assignment-1/Dataset/assignment-2-data.csv'
df_cleaned = pd.read_csv(path)

df_cleaned['new_tokenized'] = df_cleaned['clean_content'].apply(lambda x: x.split())

df_cleaned.head(5)

,id,ticker,title,category,content,release_date,provider,url,article_id,Date,Open,Close,clean_content,label,tokenized_content,new_tokenized
0,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,ECB Effects\nThe move in the euro was huge fa...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687,2019-03-07,3.77750,3.73150,ecb effect move euro huge falling pip huge imp...,0,"['ecb', 'effect', 'move', 'euro', 'huge', 'fal...","[ecb, effect, move, euro, huge, falling, pip, ..."
1,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931,2019-03-06,3.90400,3.80125,stock watch march trading session stock stock ...,0,"['stock', 'watch', 'march', 'trading', 'sessio...","[stock, watch, march, trading, session, stock,..."
2,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Investing com A rout in Apple and Facebook ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042,2018-11-19,4.04475,3.61750,investing com rout apple facebook nasdaq fb mo...,0,"['investing', 'com', 'rout', 'apple', 'faceboo...","[investing, com, rout, apple, facebook, nasdaq..."
3,221597,UBER,The Best Of CES 2020 Revised,opinion,With 4 500 companies bringing their innovation...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164,2020-01-16,6.19300,6.22325,company bringing innovation ce jan get really ...,1,"['company', 'bringing', 'innovation', 'ce', 'j...","[company, bringing, innovation, ce, jan, get, ..."
4,221614,UBER,The Best Of CES 2020,opinion,With 4 500 companies bringing their innovation...,2020-01-10,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200498063,2020-01-10,6.18325,6.10800,company bringing innovation ce jan get really ...,0,"['company', 'bringing', 'innovation', 'ce', 'j...","[company, bringing, innovation, ce, jan, get, ..."


In [22]:
# create a dictionary
dictionary = corpora.Dictionary(df_cleaned['new_tokenized'])

# Convert the list of documents (corpus) into Document Term Matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df_cleaned['new_tokenized']]

In [26]:
# split the data into train, validation and test
train = doc_term_matrix[:int(len(doc_term_matrix)*0.8)]
validation = doc_term_matrix[int(len(doc_term_matrix)*0.8):int(len(doc_term_matrix)*0.9)]
test = doc_term_matrix[int(len(doc_term_matrix)*0.9):]



In [ ]:
# Define the best coherence score
best_coherence_score = 0

# Define the best parameters
best_params = {}

# Iterate over the parameters
for num_topics in grid['num_topics']:
    for alpha in grid['alpha']:
        for eta in grid['eta']:
            for chunksize in grid['chunksize']:
                for passes in grid['passes']:
                    for iterations in grid['iterations']:
                        print(f"Training LDA model with num_topics={num_topics}, alpha={alpha}, eta={eta}, chunksize={chunksize}, passes={passes}, iterations={iterations}")
                        # Create the LDA model
                        lda_model = LdaModel(corpus=train, id2word=dictionary, num_topics=num_topics, alpha=alpha, eta=eta, chunksize=chunksize, passes=passes, iterations=iterations)
                        coherence_model = CoherenceModel(model=lda_model, texts=df_cleaned['new_tokenized'][:int(len(df_cleaned)*0.8)], dictionary=dictionary, coherence='c_v')
                        coherence_score = coherence_model.get_coherence()
                        print(f"Coherence Score: {coherence_score}")
                        if coherence_score > best_coherence_score:
                            best_coherence_score = coherence_score
                            best_params['num_topics'] = num_topics
                            best_params['alpha'] = alpha
                            best_params['eta'] = eta
                            best_params['chunksize'] = chunksize
                            best_params['passes'] = passes
                            best_params['iterations'] = iterations
                            best_params['coherence_score'] = coherence_score
                            print(f"New best coherence score: {best_coherence_score} with params: {best_params}")

# Train the best model on the combined train and validation data
best_lda_model = LdaModel(corpus=train + validation, id2word=dictionary, num_topics=best_params['num_topics'], alpha=best_params['alpha'], eta=best_params['eta'], chunksize=best_params['chunksize'], passes=best_params['passes'], iterations=best_params['iterations'])

# Evaluate the best model on the test data
coherence_model_test = CoherenceModel(model=best_lda_model, texts=df_cleaned['new_tokenized'][int(len(df_cleaned)*0.9):], dictionary=dictionary, coherence='c_v')
test_coherence_score = coherence_model_test.get_coherence()
print(f"Test Coherence Score: {test_coherence_score}")


Training LDA model with num_topics=2, alpha=symmetric, eta=symmetric, chunksize=100, passes=10, iterations=50


In [23]:



# Print the topics
for topic in lda_model.print_topics():
 print(topic)
#Get coherence score
print(CoherenceModel(model=lda_model, texts=df_cleaned['new_tokenized'], dictionary=dictionary, coherence='c_v').get_coherence())

(0, '0.012*"stock" + 0.008*"market" + 0.008*"year" + 0.006*"company" + 0.005*"earnings" + 0.005*"week" + 0.005*"nasdaq" + 0.005*"nvidia" + 0.005*"share" + 0.005*"price"')
(1, '0.013*"year" + 0.013*"company" + 0.013*"stock" + 0.012*"zacks" + 0.010*"nasdaq" + 0.010*"earnings" + 0.008*"nvidia" + 0.008*"share" + 0.008*"quarter" + 0.008*"market"')
(2, '0.013*"year" + 0.012*"nasdaq" + 0.011*"company" + 0.010*"quarter" + 0.007*"zacks" + 0.007*"stock" + 0.007*"nvidia" + 0.006*"growth" + 0.006*"million" + 0.006*"inc"')
0.37455136700082603
